In [37]:
import numpy as np
import pandas as pd


In [38]:
df = pd.read_excel("//content//crop_csv_file.xlsx")

In [39]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Temperature,humidity,soil moisture,area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,36,35,45,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,37,40,46,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,36,41,50,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,37,42,55,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,36,40,54,720.0,165.0


In [40]:
df.describe()

,Crop_Year,Temperature,humidity,soil moisture,area,Production
count,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,4.978400e+04
mean,2005.879138,34.445489,44.776116,53.110502,7363.389469,4.132019e+05
std,5.106092,3.498524,6.662406,5.258764,27927.935789,1.232609e+07
min,1997.000000,25.000000,35.000000,45.000000,0.200000,0.000000e+00
25%,2002.000000,34.000000,40.000000,50.000000,76.000000,8.700000e+01
50%,2006.000000,36.000000,42.000000,54.000000,436.000000,6.384400e+02
75%,2010.000000,36.000000,50.000000,55.000000,2546.500000,5.615000e+03
max,2014.000000,37.000000,55.000000,62.000000,877029.000000,7.801620e+08


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State_Name     49999 non-null  object 
 1   District_Name  49999 non-null  object 
 2   Crop_Year      49999 non-null  int64  
 3   Season         49999 non-null  object 
 4   Crop           49999 non-null  object 
 5   Temperature    49999 non-null  int64  
 6   humidity       49999 non-null  int64  
 7   soil moisture  49999 non-null  int64  
 8    area          49999 non-null  float64
 9   Production     49784 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 3.8+ MB


In [42]:
df['Production'].isna().sum()

215

In [43]:
df = df.dropna()

In [44]:
df.shape

(49784, 10)

In [172]:
df["Season"].unique()

array(['Kharif     ', 'Whole Year ', 'Autumn     ', 'Rabi       ',
       'Summer     ', 'Winter     '], dtype=object)

In [174]:
df["Season"] = df["Season"].str.strip()

In [177]:
df["Crop"].unique()

array(['Arecanut', 'Other Kharif pulses', 'Rice', 'Banana', 'Cashewnut',
       'Coconut ', 'Dry ginger', 'Sugarcane', 'Sweet potato', 'Tapioca',
       'Black pepper', 'Dry chillies', 'other oilseeds', 'Turmeric',
       'Maize', 'Moong(Green Gram)', 'Urad', 'Arhar/Tur', 'Groundnut',
       'Sunflower', 'Bajra', 'Castor seed', 'Cotton(lint)', 'Horse-gram',
       'Jowar', 'Korra', 'Ragi', 'Tobacco', 'Gram', 'Wheat', 'Masoor',
       'Sesamum', 'Linseed', 'Safflower', 'Onion', 'other misc. pulses',
       'Samai', 'Small millets', 'Coriander', 'Potato',
       'Other  Rabi pulses', 'Soyabean', 'Beans & Mutter(Vegetable)',
       'Bhindi', 'Brinjal', 'Citrus Fruit', 'Cucumber', 'Grapes', 'Mango',
       'Orange', 'other fibres', 'Other Fresh Fruits', 'Other Vegetables',
       'Papaya', 'Pome Fruit', 'Tomato', 'Mesta', 'Cowpea(Lobia)',
       'Lemon', 'Pome Granet', 'Sapota', 'Cabbage', 'Rapeseed &Mustard',
       'Peas  (vegetable)', 'Niger seed', 'Bottle Gourd', 'Varagu',
       'Garl

In [178]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [179]:
X = df.drop(["District_Name","Production"],axis=1)
y = df["Production"]

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [181]:
ohe = OneHotEncoder()
ohe.fit(X[["State_Name","Season","Crop"]])

OneHotEncoder()

In [182]:
categories = ohe.categories_

In [183]:
column_trans = make_column_transformer((OneHotEncoder(drop="first",sparse = False,categories=categories),["State_Name","Season","Crop"]),remainder="passthrough")

In [184]:
X_train_tnf = column_trans.fit_transform(X_train)
X_test_tnf = column_trans.transform(X_test)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tnf)
X_test_scaled = scaler.transform(X_test_tnf)



In [185]:
lr = LinearRegression()

In [186]:
pipe = make_pipeline(column_trans,lr)

In [187]:
lr.fit(X_train_scaled,y_train)

LinearRegression()

In [188]:
y_pred_lr = lr.predict(X_test_scaled)

In [189]:
from sklearn.metrics import mean_squared_error,r2_score

In [190]:
mse = mean_squared_error(y_test,y_pred_lr)

In [191]:
r2_score(y_pred_lr,y_test)

-3.636099123030392

**Random forest**

In [192]:
model = RandomForestRegressor()

In [193]:
model.fit(X_train_tnf,y_train)

RandomForestRegressor()

In [194]:
y_pred = model.predict(X_test_tnf)

In [195]:
y_pred

array([ 1038.33,  5976.09, 12267.06, ...,   226.71,   266.01, 21341.04])

In [196]:
r2_score(y_test,y_pred)

0.9033242960189548

**predict**

In [197]:
predict_data = [["Andaman and Nicobar Islands",2000,"Kharif","Arecanut",36,35,45,1254.0]]
data = {"State_Name":["Andaman and Nicobar Islands"],"Crop_Year":[2000],"Season":["Kharif"],"Crop":["Arecanut"],"Temperature":[36],"humidity":[35],"soil moisture":[45]," area":[1245.0]}
predict_df = pd.DataFrame(predict_data,columns=["State_Name","Crop_Year","Season","Crop","Temperature","humidity","soil moisture","area"])

In [198]:
pd.DataFrame(data)

,State_Name,Crop_Year,Season,Crop,Temperature,humidity,soil moisture,area
0,Andaman and Nicobar Islands,2000,Kharif,Arecanut,36,35,45,1245.0


In [199]:

tnf = column_trans.transform(pd.DataFrame(data))

In [200]:
model.predict(tnf)

array([1500.895])